# Deep Learning MTH 5320 
## Homework 3
## 12/1/2023
## By Josias Moukpe

## Object Detection Model Assignment

### Dataset Prep and Preprocessing (20 points)
- Collect or create a dataset of images containing the objects you want to detect annotated with bounding boxes.
- Implement data preprocessing techniques, including:
  - Resizing images
  - Normalizing pixels
- Hint: You can find pre-annotated datasets for download on [Roboflow](https://github.com/roboflow/supervision).

### Model Architecture (30 points)
- Train a YOLOv8 model on your selected dataset using the public repository available at [Ultralytics](https://github.com/ultralytics/ultralytics).
- Monitor the training progress and use appropriate techniques to prevent overfitting.
- Conduct at least 3 tuning experiments.

### Evaluation (20 points)
- Evaluate your trained model's performance on a test dataset using mAP (mean Average Precision).
- Visualize the model's predictions on sample images.
- Analyze the model's performance, discussing any specific failure modes on your test dataset.

### BONUS (25 bonus points)
- Design and train a custom CNN-based object detection model using a self-designed architecture.
- Alternatively, compare several publicly available models on the same dataset.
- Consider implementing Gaussian YOLO V8 and comparing it against YOLO V8.

Please ensure that your code includes docstrings and type hints as requested, and be thorough in your analysis and experimentation.

### Setup and Imports

In [1]:
# Import libraries
import torch
import torchvision
import numpy as np
from ultralytics import YOLO

# import type hints
from typing import Tuple, List, Dict, Union, Any

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')